# ETFユニバース選定

このノートブックでは、ETF版「統計的トレンド逆張り」戦略のユニバース選定プロセスを実行します。
処理時間短縮のため、サンプルETFセットを使用するオプションが含まれています。

In [ ]:
# Google Colabでの環境設定
import sys
if 'google.colab' in sys.modules:
    # 自動環境セットアップ
    print("🔄 Google Colab環境を準備中...")
    
    try:
        # 最新版のsetup_colab.pyを取得する場合はコメントを外してください
        # !wget -q https://raw.githubusercontent.com/your-username/etf-mean-reversion-strategy/main/notebooks/setup_colab.py
        %run ../notebooks/setup_colab.py
    except:
        print("⚠️ setup_colab.pyの実行に失敗しました。基本的な設定を行います。")
        from google.colab import drive
        drive.mount('/content/drive')
        %cd /content/drive/MyDrive/etf-mean-reversion-strategy
    
    # サンプルETFセットを使用するオプション（処理時間短縮のため）
    USE_SAMPLE_ETF_SET = True
    
    if USE_SAMPLE_ETF_SET:
        # サンプルETFモードに切り替え
        try:
            import src.universe.sample_etfs
            src.universe.sample_etfs.override_universe_selection('src.universe')
        except Exception as e:
            print(f"⚠️ サンプルETFモードへの切り替えに失敗しました: {str(e)}")
    
    # メモリ使用状況の確認
    try:
        display_memory_usage()
    except:
        print("⚠️ メモリ使用量モニタリングを利用できません")

In [ ]:
# 必要なライブラリとモジュールのインポート
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
import json
from datetime import datetime
from IPython.display import display, HTML

# プロジェクトモジュールのインポート
from src.data.fetch import get_base_etf_list
from src.universe import select_universe

In [ ]:
# 結果ディレクトリの作成
os.makedirs("data/results", exist_ok=True)

In [ ]:
# 実行の開始
start_time = time.time()
print(f"ETFユニバース選定を開始します: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# メモリ状態のチェックポイント
try:
    print("\nメモリ使用状況（開始時）:")
    # メモリ使用量を表示する関数を定義
    def check_memory_usage():
        try:
            import psutil
            process = psutil.Process(os.getpid())
            memory_info = process.memory_info()
            memory_usage_gb = memory_info.rss / (1024 ** 3)
            print(f"現在のメモリ使用量: {memory_usage_gb:.2f} GB")
            return memory_usage_gb
        except:
            print("メモリ使用量の確認ができません")
            return None
            
    memory_start = check_memory_usage()
except:
    memory_start = None

## ステップ1: 基礎候補リストの取得

In [ ]:
print("\n--- ステップ1: 基礎候補リストの取得 ---")
try:
    # サンプルモードが有効な場合、サンプルETFリストを使用
    if 'USE_SAMPLE_ETF_SET' in globals() and USE_SAMPLE_ETF_SET:
        import src.universe.sample_etfs
        base_etfs = src.universe.sample_etfs.get_sample_etfs()
        print(f"サンプルETFリスト: {len(base_etfs)}銘柄")
    else:
        base_etfs = get_base_etf_list()
        print(f"基礎候補リスト: {len(base_etfs)}銘柄")
    
    # デモンストレーション用に最初の5つを表示
    print("\n基礎候補リストのサンプル:")
    sample_df = pd.DataFrame(base_etfs[:5])
    display(sample_df)
except Exception as e:
    print(f"エラー: 基礎候補リストの取得に失敗しました: {str(e)}")
    import traceback
    traceback.print_exc()
    base_etfs = []

# 処理時間のチェックポイント
checkpoint_time = time.time()
print(f"基礎候補リスト取得にかかった時間: {(checkpoint_time - start_time):.2f}秒")

# メモリ状態のチェックポイント
try:
    print("\nメモリ使用状況（基礎候補リスト取得後）:")
    memory_after_base = check_memory_usage()
except:
    pass

## ステップ2〜5: ETFユニバースの選定

In [ ]:
print("\n--- ステップ2〜5: ETFユニバースの選定 ---")
try:
    # 選定開始時刻を記録
    selection_start = time.time()
    
    # ユニバース選定を実行
    final_universe = select_universe(base_etfs)
    print(f"最終ETFユニバース: {len(final_universe)}銘柄")
    
    # 選定終了時刻と処理時間を表示
    selection_end = time.time()
    selection_time = selection_end - selection_start
    print(f"ユニバース選定にかかった時間: {selection_time:.2f}秒 ({selection_time/60:.2f}分)")
    
    # メモリ状態のチェックポイント
    try:
        print("\nメモリ使用状況（ユニバース選定後）:")
        memory_after_selection = check_memory_usage()
    except:
        pass
    
    # 結果を保存
    results_path = "data/results/final_etf_universe.json"
    with open(results_path, 'w') as f:
        json.dump(final_universe, f, indent=2)
    print(f"最終ETFユニバースを保存しました: {results_path}")
    
    # 読みやすいCSV形式でも保存
    csv_path = "data/results/final_etf_universe.csv"
    df = pd.DataFrame(final_universe)
    df.to_csv(csv_path, index=False)
    print(f"CSV形式でも保存しました: {csv_path}")
    
    # 結果の表示
    if len(final_universe) > 0:
        # データフレームとして整形して表示
        df = pd.DataFrame(final_universe)
        
        # 表示する列を選択（利用可能な場合）
        display_columns = ['symbol', 'name', 'avg_volume', 'aum', 'estimated_spread', 'cluster']
        available_columns = [col for col in display_columns if col in df.columns]
        
        print("\n最終ETFユニバース:")
        display(df[available_columns])
        
        # クラスタ分布を確認
        if 'cluster' in df.columns:
            print("\nETFクラスタ分布:")
            plt.figure(figsize=(12, 6))
            cluster_counts = df['cluster'].value_counts().sort_index()
            ax = cluster_counts.plot(kind='bar')
            plt.title('ETFクラスタ分布')
            plt.xlabel('クラスタID')
            plt.ylabel('ETF数')
            for i, v in enumerate(cluster_counts):
                ax.text(i, v + 0.1, str(v), ha='center')
            plt.tight_layout()
            plt.savefig("data/results/etf_cluster_distribution.png", dpi=300, bbox_inches='tight')
            plt.show()
    else:
        print("警告: 最終ETFユニバースが空です。処理に問題があった可能性があります。")
except Exception as e:
    print(f"エラー: ETFユニバース選定に失敗しました: {str(e)}")
    import traceback
    traceback.print_exc()
    final_universe = []

## 結果のサマリー

In [ ]:
# 実行時間の表示
end_time = time.time()
execution_time = end_time - start_time
print(f"\n総処理時間: {execution_time:.2f}秒 ({execution_time/60:.2f}分)")
print(f"ETFユニバース選定が完了しました: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# 最終メモリ状態のチェック
try:
    print("\n最終メモリ使用状況:")
    memory_final = check_memory_usage()
    
    # メモリ使用の変化を表示
    if memory_start is not None and memory_final is not None:
        memory_increase = memory_final - memory_start
        print(f"メモリ使用量の増加: {memory_increase:.2f} GB")
except:
    pass

# 次のステップのガイダンス
print("\n次のステップ:")
print("1. 02_signal_generation.ipynb を実行してシグナル生成を行ってください")
print("2. 03_parameter_stability.ipynb を実行してパラメータ安定性を評価してください")

In [ ]:
# サマリーを表形式で表示
from IPython.display import HTML

summary_data = [
    ["処理ステップ", "結果"],
    ["基礎候補リスト", f"{len(base_etfs)}銘柄"],
    ["最終ETFユニバース", f"{len(final_universe)}銘柄"],
    ["処理時間", f"{execution_time:.2f}秒 ({execution_time/60:.2f}分)"],
    ["結果ファイル", f"{results_path if 'results_path' in locals() else '未保存'}"]
]

summary_html = "<table style='width:100%; border-collapse:collapse;'>"
for i, row in enumerate(summary_data):
    style = "background-color:#f2f2f2;" if i == 0 else ""
    summary_html += f"<tr style='{style}'>"
    for cell in row:
        tag = "th" if i == 0 else "td"
        summary_html += f"<{tag} style='padding:8px; border:1px solid #ddd;'>{cell}</{tag}>"
    summary_html += "</tr>"
summary_html += "</table>"

display(HTML(summary_html))

In [ ]:
# 最終結果の要約
if len(final_universe) > 0:
    # ETFの属性サマリーを作成（カテゴリや資産クラスなど）
    print("\n最終選定ETFの概要:")
    for i, etf in enumerate(final_universe[:min(10, len(final_universe))]):
        print(f"  {i+1}. {etf['symbol']}: {etf.get('name', '名称なし')}")
    
    if len(final_universe) > 10:
        print(f"  ... 他 {len(final_universe) - 10} 銘柄")